In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
from  sklearn.datasets import make_regression
import pandas as pd    
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# imports for data generator module
import sys
sys.path.append('../modules')
import mods as md

In [3]:
df=pd.read_csv('../output/dataset4.csv')
df.shape

(1000, 12)

In [4]:
df.keys()

Index(['var_0', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7',
       'target', 'id', 'race', 'race_string'],
      dtype='object')

In [5]:
df['target'].value_counts(normalize=True)

0    0.557
1    0.443
Name: target, dtype: float64

In [6]:
df_0 = df[df['target']==0]
df_0 = df_0.sample(frac = 1)
df_0 = df_0.reset_index(drop=True)
df_0.shape

(557, 12)

In [7]:
df_1 = df[df['target']==1]
df_1 = df_1.sample(frac = 1)
df_1 = df_1.reset_index(drop=True)
df_1.shape

(443, 12)

In [8]:
print(557/3)
print(443/3)

185.66666666666666
147.66666666666666


In [9]:
df_0a = df_0.iloc[0:185,:]
df_0b = df_0.iloc[185:371,:]
df_0c = df_0.iloc[371:557,:]

# print
print(df_0a.shape[0])
print(df_0b.shape[0])
print(df_0c.shape[0])

185
186
186


In [10]:
df_1a = df_1.iloc[0:148,:]
df_1b = df_1.iloc[148:295,:]
df_1c = df_1.iloc[295:443,:]

# print
print(df_1a.shape[0])
print(df_1b.shape[0])
print(df_1c.shape[0])

148
147
148


In [11]:
df_fold1 = pd.concat([df_1a, df_0a])
df_fold1 = df_fold1.reset_index(drop=True)
print(df_fold1.shape)

df_fold2 = pd.concat([df_1b, df_0b])
df_fold2 = df_fold2.reset_index(drop=True)
print(df_fold2.shape)

df_fold3 = pd.concat([df_1c, df_0c])
df_fold3 = df_fold3.reset_index(drop=True)
print(df_fold3.shape)

(333, 12)
(333, 12)
(334, 12)


In [12]:
df_fold1.to_csv('../output/dataset4_fold1.csv', index=False)
df_fold2.to_csv('../output/dataset4_fold2.csv', index=False)
df_fold3.to_csv('../output/dataset4_fold3.csv', index=False)

In [13]:
x1 = df_fold1.drop(['target', 'id', 'race', 'race_string'], 1)
y1 = df_fold1['target']

x2 = df_fold2.drop(['target', 'id', 'race', 'race_string'], 1)
y2 = df_fold2['target']

x3 = df_fold3.drop(['target', 'id', 'race', 'race_string'], 1)
y3 = df_fold3['target']

In [14]:
def append_preds(main, x_data, y_data):
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    
    logr = LogisticRegression()
    rforest = RandomForestClassifier()
    
    scimodel = [logr, rforest]
    
    df = main.copy()
    
    if (len(x_data) == 3) and (len(y_data) == 3):
        df1 = x_data[0].copy()
        df2 = x_data[1].copy()
        df3 = x_data[2].copy()
        
        dfs_x = [x_data[0], x_data[1]]
        X_train = pd.concat(dfs_x)
        y_train =  y_data[0].append(y_data[1])
        X_test =  x_data[2]
        y_test = y_data[2]
        
        for s in scimodel:
            print('now predicting with ', s)
            s.fit(X_train, y_train)
            preds = s.predict(X_test)
       
            df[type(s).__name__+'_preds'] = preds
        
        return df
    
    else:
        print('!DataframeError: expected 2 frames each, got ', len(x_data), ' for X and ', len(y_data), ' for y')

In [15]:
r1 = append_preds(main=df_fold1,
                  x_data=[x2,x3,x1],
                  y_data=[y2,y3,y1]
                 )

r2 = append_preds(main=df_fold2,
                  x_data=[x1,x3,x2],
                  y_data=[y1,y3,y2]
                 )

r3 = append_preds(main=df_fold3,
                  x_data=[x1,x2,x3],
                  y_data=[y1,y2,y3]
                 )

now predicting with  LogisticRegression()
now predicting with  RandomForestClassifier()
now predicting with  LogisticRegression()
now predicting with  RandomForestClassifier()
now predicting with  LogisticRegression()
now predicting with  RandomForestClassifier()


In [16]:
results = pd.concat([r1,r2,r3])
print(results.shape)

(1000, 14)


In [17]:
def get_accuracy(y_true, y_predict):
    from sklearn.metrics import accuracy_score
    return accuracy_score(y_true, y_predict)

def get_balanced_accuracy(y_true, y_predict):
    from sklearn.metrics import balanced_accuracy_score
    return balanced_accuracy_score(y_true, y_predict)

def assess_preds(df, target_col, group_col):
    z = df.copy()
    # select out t5 cols
    pred_cols=[]
    for col in z.keys():
        if 'pred' in col:
            pred_cols.append(col)
    print('found: ', pred_cols)
    print()
    
    unique_groups = z[group_col].unique()
    new_df = pd.DataFrame()
    new_df['groups'] = unique_groups
    
    for pr in pred_cols:
        print('accuracy for',pr, ':', round(get_accuracy(z[target_col], z[pr]), 2))
        maj = z[z[group_col]=='majority']
        maj_ratio = len(maj[maj[pr]==1])/len(maj)
        
        group_ratios=[]
        for g in z[group_col].unique():
            n = z[z[group_col]==g]
            total_g = len(n)
            total_sel = len(n[n[pr]==1])
            sr = total_sel/total_g
            air = sr/maj_ratio
            group_ratios.append(air)
        new_df[pr] = group_ratios
    print()
    return new_df

In [18]:
assess_preds(df=results, 
             target_col='target', 
             group_col='race_string'
            )

found:  ['LogisticRegression_preds', 'RandomForestClassifier_preds']

accuracy for LogisticRegression_preds : 0.64
accuracy for RandomForestClassifier_preds : 0.65



,groups,LogisticRegression_preds,RandomForestClassifier_preds
0,minority,0.411994,0.556222
1,majority,1.000000,1.000000


In [19]:
df[['var_1','var_2','var_3','var_4','var_5','var_6','var_7']].corr()

,var_1,var_2,var_3,var_4,var_5,var_6,var_7
var_1,1.000000,0.096728,-0.488299,0.015062,0.013288,0.124825,0.158567
var_2,0.096728,1.000000,-0.316842,-0.358270,0.001438,-0.613094,-0.780040
var_3,-0.488299,-0.316842,1.000000,0.787636,-0.234122,0.216120,0.201550
var_4,0.015062,-0.358270,0.787636,1.000000,-0.610233,0.132296,0.266185
var_5,0.013288,0.001438,-0.234122,-0.610233,1.000000,0.555451,0.081272
var_6,0.124825,-0.613094,0.216120,0.132296,0.555451,1.000000,0.343706
var_7,0.158567,-0.780040,0.201550,0.266185,0.081272,0.343706,1.000000


In [40]:
x1_fair = df_fold1.drop(['target', 'id', 'race', 'race_string', 'var_1','var_3','var_7','var_2'], 1)

x2_fair = df_fold2.drop(['target', 'id', 'race', 'race_string', 'var_1','var_3','var_7','var_2'], 1)

x3_fair = df_fold3.drop(['target', 'id', 'race', 'race_string', 'var_1','var_3','var_7','var_2'], 1)

In [41]:
r1_fair = append_preds(main=df_fold1,
                  x_data=[x2_fair,x3_fair,x1_fair],
                  y_data=[y2, y3, y1]
                 )

r2_fair = append_preds(main=df_fold2,
                  x_data=[x1_fair,x3_fair,x2_fair],
                  y_data=[y1, y3, y2]
                 )

r3_fair = append_preds(main=df_fold3,
                  x_data=[x1_fair,x2_fair,x3_fair],
                  y_data=[y1, y2, y3]
                 )

now predicting with  LogisticRegression()
now predicting with  RandomForestClassifier()
now predicting with  LogisticRegression()
now predicting with  RandomForestClassifier()
now predicting with  LogisticRegression()
now predicting with  RandomForestClassifier()


In [42]:
results_fair = pd.concat([r1_fair,r2_fair,r3_fair])
print(results_fair.shape)

(1000, 14)


In [43]:
assess_preds(df=results_fair, 
             target_col='target', 
             group_col='race_string'
            )

found:  ['LogisticRegression_preds', 'RandomForestClassifier_preds']

accuracy for LogisticRegression_preds : 0.63
accuracy for RandomForestClassifier_preds : 0.6



,groups,LogisticRegression_preds,RandomForestClassifier_preds
0,minority,0.810877,0.786634
1,majority,1.000000,1.000000


In [44]:
print(get_accuracy(results_fair['LogisticRegression_preds'], results['LogisticRegression_preds']))
print(get_accuracy(results_fair['RandomForestClassifier_preds'], results['RandomForestClassifier_preds']))

0.875
0.762


In [46]:
(.88+.76)/2

0.8200000000000001